In [2]:
PROJECT = !gcloud config get-value project # returns SList
PROJECT = PROJECT[0] # gets first element in list -> str
REGION = "us-central1"  
import os
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION

In [5]:
from google.cloud import bigquery
bq = bigquery.Client(project=PROJECT)

# Preprocessing

In [4]:
# 1. for each table in the dataset, change location link to our storage
# gs://gcs-public-data--labeled-patents/us_006.pdf --> gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_patents/pdf/us_006.pdf
# 2. create new col "filename"

In [6]:
query_string = """
#standardSQL
SELECT
    *
FROM
    `qwiklabs-gcp-00-373ac55d0e0a.labeled_patents.extracted_data`
"""

extracted_data = bq.query(query_string).to_dataframe()

In [29]:
extracted_data_modified = extracted_data.copy(deep=True)

In [8]:
original = "gs://gcs-public-data--labeled-patents/us_006.pdf"

In [22]:
# os.path.basename(original)

'us_006.pdf'

In [13]:
# IMPORTANT: NOT WORKING -- TODO
# import re
# pattern = r"gs:\/\/(.*?)/.*\.pdf"
# re.search(pattern, original).group(1)
# re.sub(r"gs:\/\/(.*?)/.*\.pdf", "qwiklabs-gcp-00-373ac55d0e0a/labeled_patents/pdf", original)

'gcs-public-data--labeled-patents'

In [24]:
def change_gcs_path(original):
    split_path = os.path.split(original)
    new_path = os.path.join("gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_patents/pdf" ,split_path[1])
    return new_path

def get_filename(original):
    return os.path.basename(original)

In [31]:
extracted_data_modified["gcs_path"] = extracted_data_modified["gcs_path"].apply(change_gcs_path)

In [35]:
extracted_data_modified["filename"] = extracted_data_modified["gcs_path"].apply(get_filename)

In [36]:
extracted_data_modified

,gcs_path,issuer,language,publication_date,class_international,class_us,application_number,filing_date,priority_date_eu,representative_line_1_eu,applicant_line_1,inventor_line_1,title_line_1,number,filename
0,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU,DE,29.08.018,E04H 6/12,None,18157874.1,21.02.2018,22.02.2017,Liedtke & Partner Patentanw√§lte,SHB Hebezeugbau GmbH,"VOLGER, Alexander",STEUERUNGSSYSTEM F√úR AUTOMATISCHE PARKH√ÑUSER,EP 3 366 869 A1,espacenet_de2.pdf
1,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU,DE,03.10.2018,A61B 1/00,None,18164602.7,28.03.2018,28.03.2017,"Gramm, Line, & Partner","Emmanouilidis, Nikos","Emmanouilidis, Nikos",MEDIZINISCHE VORRICHTUNG,EP 3 381 348 A1,espacenet_de78.pdf
2,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU,DE,03.10.2018,H05B 6/12,None,18165514.3,03.04.2018,30.03.2017,None,BSH Hausger√§te GmbH,"Acero Acero, Jesus",VORRICHTUNG ZUR INDUKTIVEN ENERGIE√úBERTRAGUNG,EP 3 383 141 A2,espacenet_de73.pdf
3,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU,DE,03.10.2018,H01Q 1/22,None,18164695.1,28.03.2018,31.03.2017,"Kruspig, Volkmar et al",Antennentechnik ABB Bad,"Harz, Thomas",ELEKTROTECHNISCHE EINRICHTUNG,EP 3 382 793 A1,espacenet_de77.pdf
4,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU,DE,03.10.2018,A01K 31/00,None,18171005.4,05.02.2015,05.02.2014,Stork Bamberger Patentanw√§lte,Linco Food Systems A/S,"Thrane, Uffe",MASTH√ÑHNCHENCONTAINER ALS BESTANDTEIL EINER E...,EP 3 381 276 A1,espacenet_de56.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU,FR,30.08.2017,H04L 9/32,None,17157289.4,22.02.2017,25.02.2016,Brevalex,Commissariat √† l'Energie Atomique,"SAVRY, Olivier",M√âTHODE DE GESTION DE CERTIFICATS IMPLICITES ...,EP 3 211 826 A1,espacenet_fr25.pdf
339,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU,FR,30.09.2015,E21B 43/00,None,15305280.8,24.02.2015,25.03.2014,None,IFP Energies nouvelles,"Mesri, Youssef",PROC√âD√â DE CONSTRUCTION D'UN MAILLAGE OPTIMI...,EP 2 924 232 A2,espacenet_fr61.pdf
340,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU,FR,30.12.2015,H04L 29/06,None,15173447.2,23.06.2015,27.06.2014,"Delumeau, Fran√ßois Guy et al",Orange,"He, Ruan",PROCEDE DE GESTION DE CONTROLE D'ACCES DANS UN...,EP 2 961 130 A1,espacenet_fr45.pdf
341,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU,FR,31.08.2016,G06F 19/00,None,16156915.7,23.02.2016,25.02.2015,"Delumeau, Fran√ßois Guy et al",Biomnis,"BESSON, Michel L.",SYSTEME POUR L'ANALYSE D'ECHANTILLONS POUVANT ...,EP 3 062 251 A1,espacenet_fr100.pdf


In [ ]:
# optional: Preprocess DE, FR files to substitue error characters